In [ ]:
pip install ../QueryGPT/

In [ ]:
from querygpt.model import Query, Persona

## Persona selection

to use generate queries with this package we first need to select a persona to use. We can choose to add our own Persona or just load a persona from the register.  
To add a persona we can just plug in the values to the `Persona class`

In [ ]:
jason = Persona(
    name="Jason",
    age=4,
    gender="male",
    job="child",
    goal="find something funny online",
    residence="paris",
    query_length=4,
    complexity=None
)

For now we are only going to load a persona from the register. This will give us all possible personas from the register.

In [ ]:
Persona.list_register()

To select a Persona simply use the following function.

In [ ]:
person = Persona.from_register("edith")

alternitavely a persona can also be loaded from a `.json` file with the `Persona.from_json()` method. The json file should look something like this:
```
{
    "name": "Edith",
    "age": 75,
    "gender": "female",
    "job": "retired",
    "goal": "researching background information",
    "residence": "suburbs of san diego",
    "query_length": 4,
    "complexity": null
}

```

## Query selection

After selecting a Persona we will need to build our query. As for the Personas we can also from a handfull of variants to load in the QueryTypes.  
- Query.select_query_type  --> loads from the register
- Query.add_custom_query_from_file --> loads from a text file
- Query.add_custom_query_from_folder --> loads from a folder containing text files
- Query.add_queries --> adds a query from a string and a QueryType name

In [ ]:
q = Query()

We will first look at how to add your own query and then load one from the register.

In [ ]:
query = """

Example query for $PERSONA_NAME.

"""
q.add_queries("EXAMPLE", query)

Now when we load all possible QueryTypes notice how there is one called `EXAMPLE`.

In [ ]:
q.list_query_types()

In [ ]:
q.select_querie_type("QUERY_NO_EXAMPLE")

In [ ]:
q.select_persona(person)

In [ ]:
q.try_query_mappings()

In [ ]:
q.manual_replace("$PERSONA_PROFESSION", "student")

In [ ]:
q.query

Now we load in the core nist topics

In [ ]:
import json
core_nist = json.load(open("core_nist.json"))

In [ ]:
nist = core_nist[0]
nist

We now use the core nist topic to generate a query for the specific persona + query. `narrative` and `description` are both optional. `n_queries` defaults to 5.

In [ ]:
query = q.generate_query(topic=nist["title"], description=nist["desc"], n_queries=5)
print(query)

In [ ]:
import openai
openai.api_key = ""

In [ ]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": query}
    ],
    temperature=0.2
    )

completion.choices[0].message

In [ ]:
print(completion.choices[0].message.content)

In [ ]:
answer = eval(completion.choices[0].message.content)

In [ ]:
answer